In [1]:
from openai import OpenAI

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import START, StateGraph, END

from IPython.display import Markdown, Image
from typing_extensions import TypedDict

import json
import ast

In [3]:
from dotenv import load_dotenv
load_dotenv()

# create openAI client
client = OpenAI()

In [4]:
COMPANY = 'Beat81'
LOCATION = 'DE'
POSITION = 'Data Analytics Working Student'
BACKGROUND = [
    "Former Senior Manager in Revenue Management and Analytics at Zalando",
    "Did Pro Bono at the UN",
    "Currently Founder of Data Heroes and Bitte",
]
INTERVIEWER = "Matthias Glöel" # only works for high-profile people

In [6]:
# Create a TypedDict Class to store information
class AgentState(TypedDict):
    # Initial Data
    company: str
    location: str # country, ISO Code
    position: str
    background: list
    interviewer: str

    # Generated Content
    company_data: str
    interviewer_data: str
    questions: str

    # App keys
    memory: list

In [7]:
# Researching about the company with OpenAI web search
def research_company(state):
  # Inputs for web search
  state['company'] = COMPANY
  state['location'] = LOCATION

  print('---Researching Company---')
  # Call the responses API
  response = client.responses.create(
      model = "gpt-4.1",
      tools = [{"type": "web_search_preview",
              "user_location": {
                  "type": "approximate",
                  "country": state['location']
              }}],
      input = f"Research about {state['company']}"
  )

  # Store the outcome
  state['company_data'] = response.output_text

  # display(Markdown(state['company_data']))
  return state

In [8]:
# Test the function
example_data = research_company({'company': COMPANY})
example_data['company_data']

---Researching Company---


"BEAT81 is a Berlin-based fitness technology company founded in 2017 by professional badminton player and former national champion Tim Dettmann. The company offers high-intensity interval training (HIIT) and indoor cycling workouts that integrate heart rate monitoring technology, expert coaching, and a supportive community to enhance both physical and mental well-being. ([beat81.com](https://www.beat81.com/about?utm_source=openai))\n\n**Workout Formats and Technology**\n\nBEAT81 provides 45-minute workouts across various formats, including HIIT, Strength & Cardio, Strength, Ride, and Circuit Pump. Participants wear heart rate sensors during sessions, allowing real-time performance tracking displayed on mobile BEAT81 TVs. After each workout, personalized training results are delivered via the BEAT81 app, enabling users to monitor progress and set goals effectively. ([offerzen.com](https://www.offerzen.com/companies/beat81?utm_source=openai))\n\n**Community and Accessibility**\n\nEmphasi

In [9]:
# Build the function to research about the interviewer
def research_interviewer(state):

  # Inputs for the web search
  state['company'] = COMPANY
  state['interviewer'] = INTERVIEWER
  state['location'] = LOCATION
  state['position'] = POSITION

  print('---Researching Interviewer---')
  # Call the responses API
  response = client.responses.create(
      model = "gpt-4.1",
      tools = [{"type": "web_search_preview",
              "user_location": {
                  "type": "approximate",
                  "country": state['location']
              },
                "search_context_size": "medium", # balanced scope
                }],
      input = f"Research about {state['interviewer']}, who works for {state['company']} located in {state['location']}"
  )

  # Update the AgentState
  state['interviewer_data'] = response.output_text

  # display(Markdown(state['interviewer_data']))
  return state



In [10]:
# Test the research interviewer function
example_data_interviewer = research_interviewer(example_data)
example_data_interviewer['interviewer_data']

---Researching Interviewer---


"Matthias Gloël is an Analytics Engineer at BEAT81, a fitness company based in Berlin, Germany. ([theorg.com](https://theorg.com/org/beat81/org-chart/luis-cipriani?utm_source=openai)) BEAT81 offers 45-minute high-intensity interval training (HIIT) and cycling workouts that incorporate heart rate technology, expert coaching, and a supportive community. These sessions aim to help participants train more effectively and achieve their fitness goals. ([de.beat81.com](https://de.beat81.com/?utm_source=openai))\n\nIn his role, Matthias focuses on analyzing data to enhance BEAT81's programs and improve user experiences. By interpreting workout metrics, he contributes to refining the company's offerings and ensuring that members receive personalized and effective training sessions.\n\nBEAT81, founded in 2017, has grown to operate in multiple cities across Germany, providing a range of fitness classes led by expert coaches. The company emphasizes the use of technology and data analytics to deliv

In [11]:
def generate_questions(state):
  state['background'] = BACKGROUND
  # Define system message for AI that generates questions
  developer_message = f"""
  You are an AI that generates Interview questions to help prepare the user.
  Data:
  Company name: {state['company']}
  Interviewer: {state['interviewer']}
  User background: {state['background']}
  Job position: {state['position']}
  company_report from web search: {state['company_data']}
  Interviewer report from web search: {state['interviewer_data']}

  Output:
  1) 10 concise question in bullet form to mimic a job interview.
  2) Questions may touch upon the user's background, culture fit, job fit, or qualifications.
  """

  print('---Generating Questions---')
  response = client.responses.create(
      model = "gpt-4.1",
      input=[
          {"role": "developer", "content": developer_message}],)
  state['questions'] = response.output_text

  return state

In [12]:
# Test the generate questions function
example_data_questions = generate_questions({'interviewer_data': example_data_interviewer['interviewer_data'],
                                             'company': COMPANY,
                                             'background': BACKGROUND,
                                             'position': POSITION,
                                             'company_data': example_data['company_data'],
                                             'interviewer': INTERVIEWER})
display(Markdown(example_data_questions['questions']))

---Generating Questions---


1. Can you tell me about your experience at Zalando and how it has prepared you for a data analytics role at BEAT81?  
2. What motivated you to transition from revenue management and analytics to founding your own companies?  
3. At Data Heroes and Bitte, what were some key data challenges you faced, and how did you overcome them?  
4. BEAT81 uses heart rate and workout data to personalize training—what analytics techniques would you apply to improve this user feedback loop?  
5. Our core values include inclusivity and energizing others—can you share an example of how you’ve fostered these values in prior roles?  
6. Given our emphasis on tech-driven fitness, how would you use data analytics to enhance community engagement at BEAT81?  
7. As a working student, how would you prioritize tasks and collaborate with both engineers and coaches in a fast-paced environment?  
8. How has your pro bono work at the UN influenced your approach to working in diverse and mission-driven teams?  
9. What tools, programming languages, or platforms are you most comfortable with for analytics projects, and how have you used them in recent work?  
10. BEAT81 is rapidly expanding—how would you leverage analytics to support our growth and ensure a consistent member experience across cities?